In [1]:
#installing required libraries
!pip install transformers datasets evaluate rouge_score

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=57222562821601f793e5a99e90b4f0499b538f4be3bc83af75652014663449eb
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
#importing libraries
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download("punkt")
import evaluate


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
#Function for preprocessing
def preprocess(examples):
    # Function to clean up input and output texts
    preprocess_input = lambda x: " ".join(x.lower()strip().split()) if x else ""
    return {'inputs': [preprocess_input(doc) for doc in examples['Input']],
            'outputs': [preprocess_input(doc) for doc in examples['Output']]}

# Load and preprocess the dataset
dataset = load_dataset('zqz979/meta-review')
dataset = dataset.map(preprocess, batched=True)

#removing null rows from dataset
dataset = dataset.filter(lambda x: x['inputs'] and x['outputs'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

...

Map:   0%|          | 0/1648 [00:00<?, ? examples/s]

Map:   0%|          | 0/1649 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7692 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1648 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1649 [00:00<?, ? examples/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [6]:
def tokenize_function(examples):
    start_prompt = "Summarize the following meta-review: "
    end_prompt = " Summary: "
    prompt = [start_prompt + input_text + end_prompt for input_text in examples['Input']]
    input_encodings = tokenizer(prompt, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    label_encodings = tokenizer(examples['Output'], padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    examples['input_ids'] = input_encodings['input_ids']
    examples['labels'] = label_encodings['input_ids']

    return examples

tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['inputs', 'outputs'])
tokenized_datasets = tokenized_datasets.filter(lambda examples, index: index % 100 == 0, with_indices=True)


Map:   0%|          | 0/7680 [00:00<?, ? examples/s]

Map:   0%|          | 0/1645 [00:00<?, ? examples/s]

Map:   0%|          | 0/1645 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7680 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1645 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1645 [00:00<?, ? examples/s]

In [7]:
tokenized_datasets = tokenized_datasets.remove_columns(['Input', 'Output'])

print(tokenized_datasets['train'].shape)
print(tokenized_datasets['validation'].shape)
print(tokenized_datasets['test'].shape)

(77, 2)
(17, 2)
(17, 2)


In [9]:
tokenized_datasets['train'][0].keys()

dict_keys(['input_ids', 'labels'])

In [10]:
from transformers import TrainingArguments, Trainer
training_args=TrainingArguments(
    output_dir="./result",
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    auto_find_batch_size=True,
    evaluation_strategy='epoch',
    logging_steps=10
)
trainer=Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.991400,1.697169


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=10, training_loss=1.991439437866211, metrics={'train_runtime': 146.8816, 'train_samples_per_second': 0.524, 'train_steps_per_second': 0.068, 'total_flos': 83433527181312.0, 'train_loss': 1.991439437866211, 'epoch': 1.0})

In [29]:
sample=dataset['train']['Input'][0]

In [30]:
given=dataset['train']['Output'][0]

In [23]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def generate_summary(text, llm, tokenizer=None):
    """Generate a summary of the given meta review in a clear and concise manner.
    """
    if tokenizer is None:
        # Assuming you're using the same tokenizer for loading and generating
        tokenizer = AutoTokenizer.from_pretrained(llm.config._name_or_path)

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(llm.device)

    summary_ids = llm.generate(**inputs, max_length=150, min_length=40, num_beams=4, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

loaded_model = AutoModelForSeq2SeqLM.from_pretrained("/content/result/checkpoint-10/")
output = generate_summary(sample, llm=loaded_model)

In [24]:
print(output)

In this paper, the author investigates how to utilize large-scale human video to train dexterous robot manipulation skills. The authors perform experiments with multiple real-world tasks with pick and place, pushing, and rotating objects.


In [31]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(given, output)

# Print the scores
print(scores)

{'rouge1': Score(precision=0.5277777777777778, recall=0.2923076923076923, fmeasure=0.3762376237623763), 'rouge2': Score(precision=0.14285714285714285, recall=0.078125, fmeasure=0.101010101010101), 'rougeL': Score(precision=0.2777777777777778, recall=0.15384615384615385, fmeasure=0.198019801980198)}


In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize variables to store total scores
total_rouge1 = 0
total_rouge2 = 0
total_rougeL = 0
num_samples = 0

# Iterate over the dataset
for i in range(len(dataset['train'])):
    sample = dataset['train']['Input'][i]
    reference_summary = dataset['train']['Output'][i]

    # Generate summary
    generated_summary = generate_summary(sample, llm=loaded_model)

    # Calculate ROUGE scores for the current sample
    scores = scorer.score(reference_summary, generated_summary)

    # Accumulate scores
    total_rouge1 += scores['rouge1'].fmeasure
    total_rouge2 += scores['rouge2'].fmeasure
    total_rougeL += scores['rougeL'].fmeasure
    num_samples += 1

# Calculate average scores
avg_rouge1 = total_rouge1 / num_samples
avg_rouge2 = total_rouge2 / num_samples
avg_rougeL = total_rougeL / num_samples

# Print average scores
print(f"Average ROUGE-1: {avg_rouge1:.4f}")
print(f"Average ROUGE-2: {avg_rouge2:.4f}")
print(f"Average ROUGE-L: {avg_rougeL:.4f}")